In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import string

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['dev_server']
sladb = config['db']['sladb']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + sladb + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Write all of the sql queries required for the tests

In [5]:
sql_count_sla_id = 'select count(*) from sladb.dbo.tbl_ref_sla'

In [6]:
sql_distinct_sla_id = 'select distinct sla_id from sladb.dbo.tbl_ref_sla'

In [7]:
# sql_sla_code = '''select * from \
# sladb.dbo.tbl_ref_sla_code'''

In [8]:
sql_sla_code = '''select min(sla_code) as min_sla_code, 
                          max(sla_code) as max_sla_code, 
                          count(*) as n_sla_codes
                   from sladb.dbo.tbl_ref_sla_code'''

In [9]:
sql_date_category = 'select distinct(date_category_id) from sladb.dbo.tbl_ref_sla_season_category'

In [10]:
sql_sla_translation = 'select sla_id, sla_code, date_category_id from sladb.dbo.tbl_ref_sla_translation'

## Execute all of the sql queries and store the results

In [11]:
before_count_sla_id = pd.read_sql(con = engine, sql = sql_count_sla_id).values[0][0]

In [12]:
before_count_sla_id

8

In [13]:
before_squared_count_sla_id = before_count_sla_id * before_count_sla_id

In [14]:
before_squared_count_sla_id

64

In [15]:
before_distinct_sla_id = pd.read_sql(con = engine, sql = sql_distinct_sla_id)

In [16]:
before_distinct_sla_id

,sla_id
0,A
1,B
2,C
3,D
4,E
5,F
6,N
7,T


In [17]:
before_sla_code = pd.read_sql(con = engine, sql = sql_sla_code)

In [18]:
before_sla_code

,min_sla_code,max_sla_code,n_sla_codes
0,1,64,64


In [19]:
before_date_category = pd.read_sql(con = engine, sql = sql_date_category)

In [20]:
before_date_category

,date_category_id
0,1
1,2


In [21]:
before_sla_translation = pd.read_sql(con = engine, sql = sql_sla_translation)

In [22]:
before_sla_translation

,sla_id,sla_code,date_category_id
0,A,1,1
1,A,2,1
2,A,3,1
3,A,4,1
4,A,17,1
...,...,...,...
123,T,28,2
124,T,30,2
125,T,36,2
126,T,47,2


In [581]:
timing = []

In [582]:
descriptions = []

In [583]:
results = []

## Before Tests

### First before test

In [584]:
timing.append('before')

In [585]:
descriptions.append('''Check that the count of sla_ids (from tbl_ref_sla) squared \
is equal to the number of sla_codes in tbl_ref_sla_code''')

In [586]:
results.append(before_squared_count_sla_id == before_sla_code['n_sla_codes'].values[0])

### Second before test

In [587]:
timing.append('before')

In [588]:
descriptions.append('''Test that the number of sla_codes in tbl_ref_sla_code \
is equal to the maximum sla_code in tbl_ref_sla_code''')

In [589]:
results.append(before_sla_code['n_sla_codes'].values[0] == before_sla_code['max_sla_code'].values[0])

### Third before test

In [590]:
timing.append('before')

In [591]:
descriptions.append('''Test that the minimum sla_code in tbl_ref_sla_code is equal to 1''')

In [592]:
results.append(1 == before_sla_code['min_sla_code'].values[0])

### Fourth before test

In [593]:
timing.append('before')

In [594]:
descriptions.append('''Test that the count of records in tbl_ref_sla_translation \
is equal to the count of sla_codes in tbl_ref_sla_code multiplied by the number \
of date_category_ids in tbl_ref_sla_season_category''')

In [595]:
results.append(
    len(before_sla_translation) == (before_sla_code['n_sla_codes'].values[0] *
                                    len(before_date_category)))

### Fifth before test

In [596]:
timing.append('before')

In [597]:
descriptions.append('''Test the uniqueness of the rows in tbl_ref_sla_translation''')

In [598]:
before_sla_translation_nodup = before_sla_translation.drop_duplicates()

In [599]:
results.append(len(before_sla_translation) == len(before_sla_translation_nodup))

### Sixth before test

In [600]:
timing.append('before')

In [601]:
descriptions.append(
    '''Test that all sla_ids, sla_codes and date_category_ids have been \
    accounted for in tbl_ref_sla_translation''')

In [602]:
before_unique_sla_translation = before_sla_translation.nunique()

In [603]:
results.append(
    ((before_unique_sla_translation['sla_id'] == before_count_sla_id)
     and (before_unique_sla_translation['date_category_id'] == len(
         before_date_category))
     and (before_unique_sla_translation['sla_code'] ==
          before_sla_code['n_sla_codes'].values[0])))

## Insert the sla_id values into tbl_ref_sla so that the after tests can be conducted

In [604]:
ninserts = 2

In [605]:
sla_id = [x for x in string.ascii_uppercase if x not in(before_distinct_sla_id.values)]

In [606]:
sla_inserts = [[x, 'Testing new SLA: '+ x, 1, 2] for x in sla_id]

In [607]:
sla_inserts_df = pd.DataFrame(sla_inserts[0:ninserts], 
                              columns = ['sla_id', 'sla_desc', 'sla_min_days', 'sla_max_days'])

In [608]:
sla_inserts_df

,sla_id,sla_desc,sla_min_days,sla_max_days
0,G,Testing new SLA: G,1,2
1,H,Testing new SLA: H,1,2


In [609]:
# sla_inserts_df.to_sql('tbl_ref_sla', con = engine, index = False, if_exists = 'append')

## Execute all of the sql queries AFTER INSERT and store the results

In [610]:
after_count_sla_id = pd.read_sql(con = engine, sql = sql_count_sla_id).values[0][0]

In [611]:
after_count_sla_id

10

In [612]:
after_squared_count_sla_id = (after_count_sla_id)**2 

In [613]:
after_squared_count_sla_id

100

In [614]:
after_distinct_sla_id = pd.read_sql(con = engine, sql = sql_distinct_sla_id)

In [615]:
after_distinct_sla_id

,sla_id
0,A
1,B
2,C
3,D
4,E
5,F
6,G
7,H
8,N
9,T


In [616]:
after_sla_code = pd.read_sql(con = engine, sql = sql_sla_code)

In [617]:
after_sla_code

,min_sla_code,max_sla_code,n_sla_codes
0,1,100,100


In [618]:
after_date_category = pd.read_sql(con = engine, sql = sql_date_category)

In [619]:
# after_date_category

In [620]:
after_sla_translation = pd.read_sql(con = engine, sql = sql_sla_translation)

In [621]:
# after_sla_translation

## After Tests

### First after test

In [622]:
timing.append('after')

In [623]:
descriptions.append('''Check that the count of sla_ids (from tbl_ref_sla) squared \
is equal to the number of sla_codes in tbl_ref_sla_code''')

In [624]:
results.append(after_squared_count_sla_id == after_sla_code['n_sla_codes'].values[0])

In [625]:
results

[True, True, True, True, True, True, True]

### Second after test

In [626]:
timing.append('after')

In [627]:
descriptions.append('''Test that the number of sla_codes in tbl_ref_sla_code \
is equal to the maximum sla_code in tbl_ref_sla_code''')

In [628]:
results.append(after_sla_code['n_sla_codes'].values[0] == after_sla_code['max_sla_code'].values[0])

### Third after test

In [629]:
timing.append('after')

In [630]:
descriptions.append('''Test that the minimum sla_code in tbl_ref_sla_code is equal to 1''')

In [631]:
results.append(1 == after_sla_code['min_sla_code'].values[0])

### Fourth after test

In [632]:
timing.append('after')

In [633]:
descriptions.append(
    '''Test that the count of records in tbl_ref_sla_translation is \
    equal to the count of sla_codes in tbl_ref_sla_code multiplied by \
    the number of date_category_ids in tbl_ref_sla_season_category''')

In [634]:
results.append(len(after_sla_translation) == (after_sla_code['n_sla_codes'].values[0] * len(after_date_category)))

### Fifth before test

In [635]:
timing.append('after')

In [636]:
descriptions.append('''Test the uniqueness of the rows in tbl_ref_sla_translation''')

In [637]:
after_sla_translation_nodup = after_sla_translation.drop_duplicates()

In [638]:
results.append(len(after_sla_translation) == len(after_sla_translation_nodup))

### Sixth after test

In [639]:
timing.append('after')

In [640]:
descriptions.append(
    '''Test that all sla_ids, sla_codes and date_category_ids have been \
    accounted for in tbl_ref_sla_translation''')

In [641]:
after_unique_sla_translation = after_sla_translation.nunique()

In [642]:
results.append(
    ((after_unique_sla_translation['sla_id'] == after_count_sla_id)
     and (after_unique_sla_translation['date_category_id'] == len(
         after_date_category)) and (after_unique_sla_translation['sla_code'] ==
                                    after_sla_code['n_sla_codes'].values[0])))

### Seventh after test

In [643]:
timing.append('after')

In [644]:
m = ninserts
n = before_count_sla_id

In [645]:
descriptions.append(
    '''Test that the number of rows insrted into tbl_ref_sla_code is equal to (m+n)**2-n**2 '''
)

In [646]:
results.append(after_sla_code.n_sla_codes.values[0] -
               before_sla_code.n_sla_codes.values[0] == (m + n)**2 - n**2)

### Eighth after test

In [647]:
timing.append('after')

In [648]:
descriptions.append(
    '''Test that the nmuber of rows in tbl_ref_sla_translation is equal to 2*(m+n)**2 '''
)

In [649]:
num_date_category_ids = len(after_date_category)

In [650]:
results.append(len(after_sla_translation) == num_date_category_ids * (m + n)**2)

### Ninth after test

In [651]:
timing.append('after')

In [652]:
descriptions.append('''After removing the original rows, \
test the uniqueness of the rows in tbl_ref_sla_translation''')

In [653]:
after_sla_translation_nodup = after_sla_translation.drop_duplicates()

In [654]:
results.append(len(after_sla_translation) == len(after_sla_translation_nodup))

### Tenth after test

In [655]:
timing.append('after')

In [656]:
descriptions.append(
    '''Ensuring the correct number of rows was inserted into tbl_ref_sla_trnalsation'''
)

In [657]:
added_rows_df = pd.merge(after_sla_translation,
         before_sla_translation,
         on=['sla_id', 'sla_code', 'date_category_id'],
         indicator=True,
         how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [658]:
results.append(len(added_rows_df)==(len(after_sla_translation) - len(before_sla_translation)))

## List of tuples with final test results

In [659]:
list(zip(timing, descriptions, results))

[('before',
  'Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of sla_codes in tbl_ref_sla_code',
  True),
 ('before',
  'Test that the number of sla_codes in tbl_ref_sla_code is equal to the maximum sla_code in tbl_ref_sla_code',
  True),
 ('before',
  'Test that the minimum sla_code in tbl_ref_sla_code is equal to 1',
  True),
 ('before',
  'Test that the count of records in tbl_ref_sla_translation is equal to the count of sla_codes in tbl_ref_sla_code multiplied by the number of date_category_ids in tbl_ref_sla_season_category',
  True),
 ('before',
  'Test the uniqueness of the rows in tbl_ref_sla_translation',
  True),
 ('before',
  'Test that all sla_ids, sla_codes and date_category_ids have been     accounted for in tbl_ref_sla_translation',
  True),
 ('after',
  'Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of sla_codes in tbl_ref_sla_code',
  True),
 ('after',
  'Test that the number of sla_codes in tbl_ref

In [660]:
pd.DataFrame(list(zip(timing, descriptions, results)))             

,0,1,2
0,before,Check that the count of sla_ids (from tbl_ref_...,True
1,before,Test that the number of sla_codes in tbl_ref_s...,True
2,before,Test that the minimum sla_code in tbl_ref_sla_...,True
3,before,Test that the count of records in tbl_ref_sla_...,True
4,before,Test the uniqueness of the rows in tbl_ref_sla...,True
5,before,"Test that all sla_ids, sla_codes and date_cate...",True
6,after,Check that the count of sla_ids (from tbl_ref_...,True
7,after,Test that the number of sla_codes in tbl_ref_s...,True
8,after,Test that the minimum sla_code in tbl_ref_sla_...,True
9,after,Test that the count of records in tbl_ref_sla_...,True
